# 🚀 11 - Progetto finale: Gestione di un inventario avanzato

---
## Introduzione al progetto

In questo capitolo, metterai insieme tutti i concetti imparati finora per creare un piccolo programma che gestisce un inventario di prodotti. Questa volta, useremo un approccio più professionale e modulare, sfruttando in particolare la **Programmazione Orientata agli Oggetti** per modellare il problema in modo più efficace.

Utilizzeremo:

- **Ambienti virtuali (`venv`)** per isolare le dipendenze del progetto.
- **Classi e Oggetti (OOP)** per rappresentare i prodotti e l'inventario stesso.
- **Librerie esterne** come `colorama` per migliorare l'interfaccia utente.
- **Strutture dati** (dizionari) per organizzare l'inventario all'interno di una classe.
- **Cicli (`while`, `for`)** per l'interazione con l'utente e l'elaborazione dei dati.
- **Gestione degli errori (`try-except`)** per rendere il programma robusto.
- **Input/Output da file** con il modulo `json` per salvare e caricare i dati in modo persistente.

## 0. Preparazione del Progetto con Ambienti Virtuali 💻

Prima di scrivere una riga di codice, la buona pratica ci insegna a creare un ambiente isolato per il nostro progetto. Questo ci assicura che le dipendenze non entrino in conflitto con altri progetti sul tuo sistema.

Apri il terminale, naviga nella cartella del progetto ed esegui i seguenti comandi:

1.  **Crea l'ambiente virtuale:**
    `python3 -m venv venv`

2.  **Attiva l'ambiente virtuale:**
    *Su macOS / Linux:* `source venv/bin/activate`
    *Su Windows:* `venv\Scripts\activate.bat`

3.  **Installa la libreria esterna `colorama`:**
    `pip install colorama`

Ora sei pronto per scrivere il codice!

---
## 1. Modellazione con la Programmazione Orientata agli Oggetti (OOP) 📦

Invece di usare un dizionario per rappresentare un prodotto, definiamo una **classe `Prodotto`**. Questo ci permette di raggruppare i dati (`nome`, `prezzo`, `quantita`) e le funzionalità correlate in un'unica entità logica.

In [ ]:
class Prodotto:
    """Classe per rappresentare un singolo prodotto nell'inventario."""
    def __init__(self, nome: str, prezzo: float, quantita: int):
        self.nome = nome
        self.prezzo = prezzo
        self.quantita = quantita

    def __str__(self):
        """Restituisce una rappresentazione in stringa dell'oggetto."""
        return f"Prodotto: {self.nome.capitalize()} | Prezzo: {self.prezzo:.2f}€ | Quantità: {self.quantita}"

    def to_dict(self):
        """Converte l'oggetto in un dizionario per il salvataggio."""
        return self.__dict__

    @classmethod
    def from_dict(cls, data):
        """Crea un oggetto Prodotto da un dizionario."""
        return cls(**data)

---
## 2. Gestione dell'Inventario come Classe 🏢

Ora creiamo una seconda classe, `GestoreInventario`, che si occuperà di tutte le operazioni. I metodi di questa classe incapsuleranno la logica per aggiungere, vendere, salvare e caricare i prodotti.

In [ ]:
import json
from colorama import Fore, Style

class GestoreInventario:
    """Classe che gestisce tutte le operazioni sull'inventario."""
    NOME_FILE = "inventario.json"
    
    def __init__(self):
        self.inventario = {}
        self.carica_inventario()

    def aggiungi_prodotto(self, nome: str, prezzo: float, quantita: int):
        if nome in self.inventario:
            print(f"{Fore.YELLOW}Attenzione: Il prodotto {nome} esiste già.{Style.RESET_ALL}")
        else:
            self.inventario[nome] = Prodotto(nome, prezzo, quantita)
            print(f"{Fore.GREEN}Prodotto {nome} aggiunto con successo.{Style.RESET_ALL}")

    def vendi_prodotto(self, nome: str, quantita_venduta: int):
        if nome in self.inventario:
            prodotto = self.inventario[nome]
            if prodotto.quantita >= quantita_venduta:
                prodotto.quantita -= quantita_venduta
                print(f"{Fore.GREEN}{quantita_venduta} {nome} vendute. Rimangono {prodotto.quantita}.{Style.RESET_ALL}")
            else:
                print(f"{Fore.RED}Errore: Quantità insufficiente. Disponibili: {prodotto.quantita}{Style.RESET_ALL}")
        else:
            print(f"{Fore.RED}Errore: Il prodotto {nome} non esiste.{Style.RESET_ALL}")

    def visualizza_inventario(self):
        print("\n--- Inventario attuale ---")
        if not self.inventario:
            print("L'inventario è vuoto.")
        for prodotto in self.inventario.values():
            print(prodotto)
        print("--------------------------")

    def salva_inventario(self):
        try:
            with open(self.NOME_FILE, 'w') as f:
                inventario_da_salvare = {nome: prodotto.to_dict() for nome, prodotto in self.inventario.items()}
                json.dump(inventario_da_salvare, f, indent=4)
            print(f"{Fore.GREEN}Inventario salvato con successo.{Style.RESET_ALL}")
        except IOError:
            print(f"{Fore.RED}Errore durante il salvataggio del file.{Style.RESET_ALL}")

    def carica_inventario(self):
        try:
            with open(self.NOME_FILE, 'r') as f:
                inventario_caricato = json.load(f)
                self.inventario = {nome: Prodotto.from_dict(dati) for nome, dati in inventario_caricato.items()}
            print(f"{Fore.GREEN}Inventario caricato con successo.{Style.RESET_ALL}")
        except (IOError, json.JSONDecodeError):
            print(f"{Fore.YELLOW}Nessun inventario esistente o file corrotto, ne creo uno nuovo.{Style.RESET_ALL}")
            self.inventario = {}

---
## 3. L'Interfaccia Utente e il Loop Principale 🧠

La logica di interazione con l'utente rimane la stessa, ma ora non dovremo più preoccuparci di passare l'inventario tra le funzioni. Invece, creeremo un'istanza del nostro `GestoreInventario` e chiameremo i suoi metodi, rendendo il codice più pulito e gestibile.

In [ ]:
def menu():
    """Mostra il menu e gestisce l'input dell'utente."""
    gestore = GestoreInventario()
    while True: # Ciclo infinito per l'interazione
        gestore.visualizza_inventario()
        print("\nOpzioni: (a)ggiungi, (v)endi, (s)alva, (r)icarica, (q)uit")
        scelta = input("Scegli un'opzione: ").lower()

        if scelta == 'a':
            nome = input("Nome prodotto: ")
            try: # Gestione errori per l'input di prezzo e quantità
                prezzo = float(input("Prezzo: "))
                quantita = int(input("Quantità: "))
                if prezzo <= 0 or quantita <= 0:
                    print(f"{Fore.RED}Errore: Prezzo e quantità devono essere maggiori di zero.{Style.RESET_ALL}")
                else:
                    gestore.aggiungi_prodotto(nome, prezzo, quantita)
            except ValueError:
                print(f"{Fore.RED}Errore: Prezzo e quantità devono essere numeri validi.{Style.RESET_ALL}")
        
        elif scelta == 'v':
            nome = input("Nome prodotto da vendere: ")
            try: # Gestione errori per l'input di vendita
                quantita_venduta = int(input("Quantità da vendere: "))
                if quantita_venduta <= 0:
                    print(f"{Fore.RED}Errore: La quantità da vendere deve essere maggiore di zero.{Style.RESET_ALL}")
                else:
                    gestore.vendi_prodotto(nome, quantita_venduta)
            except ValueError:
                print(f"{Fore.RED}Errore: La quantità deve essere un numero intero.{Style.RESET_ALL}")

        elif scelta == 's':
            gestore.salva_inventario()
        
        elif scelta == 'r':
            gestore.carica_inventario()

        elif scelta == 'q':
            print("Grazie, programma terminato.")
            break

        else:
            print(f"{Fore.RED}Opzione non valida, riprova.{Style.RESET_ALL}")

if __name__ == "__main__":
    menu()